In [8]:
import numpy as np
import pandas as pd
import sympy as sym
from sympy import symbols, simplify, lambdify
from sympy.vector import CoordSys3D, matrix_to_vector
from matplotlib import pyplot as plt
from IPython.display import display



def pol2cart(r, tet, phi):
    x = r*np.sin(tet)*np.cos(phi)
    y = r*np.sin(tet)*np.sin(phi)
    z = r*np.cos(tet)
    return x,y,z

def cart2pol(vec):
    x = vec[0]
    y = vec[1]
    z = vec[2]
    r = np.linalg.norm(vec)
    tet = np.arccos(z/r)
    phi = np.arctan(y/x)
    return r, tet, phi

def r_ort(tet, phi):
    x = np.sin(tet)*np.cos(phi)
    y = np.sin(tet)*np.sin(phi)
    z = np.cos(tet)
    return np.array([x, y, z])

def tet_ort(tet, phi):
    x = np.cos(tet)*np.cos(phi)
    y = np.cos(tet)*np.sin(phi)
    z = -np.sin(tet)
    return np.array([x, y, z])

def phi_ort(tet, phi):
    x = -np.sin(phi)
    y = np.cos(phi)
    z = 0
    return np.array([x, y, z])


In [5]:
class StationarMaterial:
    def __init__(self, Ms, gamma=5e8, alpha=0.018, Ku=6100.0, t=5e-9):
        self.Ms = Ms
        self.gamma = gamma
        self.alpha = alpha
        self.Ku = Ku
        self.t = t

        
class Material:
    def __init__(self, stationar):
        self.st = stationar
        
        self.Ms_l = lambda t: self.st.Ms
        self.gamma_l = lambda t: self.st.gamma
        self.alpha_l = lambda t: self.st.alpha
        self.Ku_l = lambda t: self.st.Ku
        self.t_l = lambda t: self.st.t
    
    def set_Ku_lambda(Ku_lambda):
        self.Ku_l = Ku_lambda

        
class LayeredFilm:
    def __init__(self, layer1, layer2, J):
        self.l1 = layer1
        self.l2 = layer2
        self.J = J

In [6]:
mu0 = 1.2566370621219e-6

uB_SI = 9.274009994e-24 #J/T
uB_CGS = 9.274009994e-21 #erg/G

hb_SI = 1.054571817e-34 # J*s
hb_CGS = 1.054571817e-27 #erg*s

g0_SI = uB_SI/hb_SI #1/(T*s)
g0_CGS = uB_CGS/hb_CGS #1/(G*s)


#estimate values from hysteresis loop
Ms_hyst = 800.
Ku1_hyst = 2*np.pi*(Ms_hyst**2) + 0.78e4*Ms_hyst/2.
Ku2_hyst = 2*np.pi*(Ms_hyst**2) + 0.78e4*Ms_hyst/2.
J_hyst = ((1.33-0.78)*1e4 * Ms_hyst/2.)*12e-7

layer1 = Material(
    StationarMaterial(
        Ms=Ms_hyst, 
        gamma=2.2*g0_CGS, alpha=0.1,
        Ku=Ku1_hyst,
        t=12e-7
    )
)
layer2 = Material(
    StationarMaterial(
        Ms=Ms_hyst, 
        gamma=2.2*g0_CGS, alpha=0.1,
        Ku=Ku2_hyst,
        t=12e-7
    )
)
film_inst = LayeredFilm(layer1, layer2, J_hyst)

In [18]:
from scipy import optimize

class StationaryState:
    def __init__(self):
        S = CoordSys3D('S')
        Mx1, My1, Mz1 = symbols("M_x1 M_y1 M_z1")
        Mx2, My2, Mz2 = symbols("M_x2 M_y2 M_z2")
        M1 = Mx1*S.i + My1*S.j + Mz1*S.k
        M2 = Mx2*S.i + My2*S.j + Mz2*S.k
        
        Hx, Hy, Hz = symbols("H_x H_y H_z")
        H = Hx*S.i + Hy*S.j + Hz*S.k
        
        Ku1, Ku2, J = symbols("K_u1 K_u2 J")
        t1, t2 = symbols("t1 t2")
        
        self.Ze1 = -H.dot(M1)
        self.Ze2 = -H.dot(M2)
        
        self.SW_D1 = Ku1*(Mx1**2 + My1**2)/(Mx1**2 + My1**2 + Mz1**2) + 2*sym.pi*(Mz1**2)
        self.SW_D2 = Ku2*(Mx2**2 + My2**2)/(Mx2**2 + My2**2 + Mz2**2) + 2*sym.pi*(Mz2**2)
        
        self.Ev1 = self.Ze1 + self.SW_D1
        self.Ev2 = self.Ze2 + self.SW_D2
        
        self.Es = J*M1.dot(M2)/(M1.magnitude()*M2.magnitude()) + t1*self.Ev1 + t2*self.Ev2
        
        
        
        Ze1_varset = (
            Mx1, My1, Mz1,
            Hx, Hy, Hz
        )
        self.Ze1_lamb = lambdify(Ze1_varset, self.Ze1, 'numpy')
        
        Ze2_varset = (
            Mx2, My2, Mz2,
            Hx, Hy, Hz
        )
        self.Ze2_lamb = lambdify(Ze2_varset, self.Ze2, 'numpy')
        
        
        
        SW_D1_varset = (
            Mx1, My1, Mz1,
            Ku1
        )
        self.SW_D1_lamb = lambdify(SW_D1_varset, self.SW_D1, 'numpy')
        
        SW_D2_varset = (
            Mx2, My2, Mz2,
            Ku2
        )
        self.SW_D2_lamb = lambdify(SW_D2_varset, self.SW_D2, 'numpy')
        
        
        Ev1_varset = (
            Mx1, My1, Mz1,
            Hx, Hy, Hz,
            Ku1
        )
        
        self.Ev1_lamb = lambdify(Ev1_varset, self.Ev1, 'numpy')
        
        Ev2_varset = (
            Mx2, My2, Mz2,
            Hx, Hy, Hz,
            Ku2
        )
        
        self.Ev2_lamb = lambdify(Ev2_varset, self.Ev2, 'numpy')
        
        
        Es_varset = (
            Mx1, My1, Mz1,
            Mx2, My2, Mz2,
            Hx, Hy, Hz,
            Ku1, Ku2,
            J,
            t1, t2
        )
        
        self.Es_lamb = lambdify(Es_varset, self.Es, 'numpy')
    
    
    def get_angles(
        self,
        film,
        Hext,
        tol = None,
        N = 10
    ):  
        func_l = lambda x: self.funcmin(x, film, Hext)
        
        best_res = 0
        min_E = 1e10
        for i in range(N):
            res = optimize.minimize(
                func_l, 
                x0=np.random.uniform(size=(4,))*np.array([np.pi, 2*np.pi, np.pi, 2*np.pi]), 
                #x0=np.array(0.0, 0.0, np.pi, 0.0)
                bounds=[(0,2*np.pi),(0,2*2*np.pi),(0,2*np.pi),(0,2*2*np.pi)], 
                tol = tol
            )
            
            if func_l(res.x)<min_E:
                best_res = res
                min_E = func_l(res.x)
        
        #print(best_res)
        return best_res.x
    
    
    def get_angles2(
        self,
        film,
        Hext
    ):
        ang0_diap = [0., np.pi]
        ang1_diap = [0., np.pi]
        
        ind = []
        ang0 = []
        ang1 = []
        
        for kk in range(k):
            ang0 = np.linspace(ang0_diap[0], ang0_diap[1], N)
            ang1 = np.linspace(ang1_diap[0], ang1_diap[1], N)
            
            Es_val = np.zeros((N, N))
            for i in range(len(ang1)):
                Es_val[i] = self.Es_lamb(
                    M1, M2,
                    H, phi, 
                    film.l1.t, film.l2.t,
                    Ku1, Ku2, film.J,
                    ang0[i], ang1
                )
            
            ind = np.unravel_index(Es_val.argmin(), Es_val.shape)
            
            ang0_diap = [ang0[max(ind[0]-2, 0)], ang0[min(ind[0]+2, N-1)]]
            ang1_diap = [ang1[max(ind[1]-2, 0)], ang1[min(ind[1]+2, N-1)]]
        
        res = [ang0[ind[0]], ang1[ind[1]]]
        return np.array(res)
    
        
    def funcmin(self, x, film, Hext):
        tet1 = x[0]
        phi1 = x[1]
        M1 = pol2cart(film.l1.st.Ms, tet1, phi1)
    
        tet2 = x[2]
        phi2 = x[3]
        M2 = pol2cart(film.l2.st.Ms, tet2, phi2)
        
        return self.Es_lamb(
            M1[0], M1[1], M1[2],
            M2[0], M2[1], M2[2],
            Hext[0], Hext[1], Hext[2],
            film.l1.st.Ku, film.l2.st.Ku,
            film.J,
            film.l1.st.t, film.l2.st.t        
        )
        
        
    def plot_energy(
        self, 
        film,
        Hext,
        N = 1000,
        
        ang0_diap_grad = np.array([0., 180.]),
        ang1_diap_grad = np.array([0., 180.]),
        sign = 1
    ):  
        ang0_diap_rad = np.deg2rad(ang0_diap_grad)
        ang1_diap_rad = np.deg2rad(ang1_diap_grad)
        
        ang1 = np.linspace(ang0_diap_rad[0], ang0_diap_rad[1], N)
        ang2 = np.linspace(ang1_diap_rad[0], ang1_diap_rad[1], N)
        
        
        Es_val = np.zeros((N, N))
        for i in range(len(ang1)):
            M1 = pol2cart(film.l1.st.Ms, ang1[i], 0.)
            M2 = pol2cart(film.l2.st.Ms, ang2, 0.)
            Es_val[i] = self.Es_lamb(
                M1[0], M1[1], M1[2],
                M2[0], M2[1], M2[2],
                Hext[0], Hext[1], Hext[2],
                film.l1.st.Ku, film.l2.st.Ku,
                film.J,
                film.l1.st.t, film.l2.st.t
            )
        
        fig, ax = plt.subplots()
        fig.set_figheight(6)
        fig.set_figwidth(6)
        ax.imshow(sign * np.log(Es_val - Es_val.min() + 1e-5))
        fig.tight_layout()
        
        ticks = np.arange(0, N + 1, N//5)
        ticks[-1]-=1
        ax.set_xticks(ticks)
        ax.set_yticks(ticks)
        
        ax.set_xticklabels(np.linspace(ang0_diap_grad[0], ang0_diap_grad[1], 6))
        ax.set_yticklabels(np.linspace(ang1_diap_grad[0], ang1_diap_grad[1], 6))      
        
st_state = StationaryState()

In [22]:
%matplotlib notebook

N = 1000
Hs_abs = np.linspace(0.5e3, 9.0e3, N)

print(Hs_abs)
ang1 = []
ang2 = []

for H_abs in Hs_abs:
    H_cur = pol2cart(H_abs, np.deg2rad(63.), 0.)
    st_angs = st_state.get_angles(film_inst, H_cur, N=100)
    ang1.append(min(st_angs[0], st_angs[2]))
    ang2.append(max(st_angs[0], st_angs[2]))

plt.plot(Hs_abs, ang1)
plt.plot(Hs_abs, ang2)
plt.grid(True)


[ 500.          508.50850851  517.01701702  525.52552553  534.03403403
  542.54254254  551.05105105  559.55955956  568.06806807  576.57657658
  585.08508509  593.59359359  602.1021021   610.61061061  619.11911912
  627.62762763  636.13613614  644.64464464  653.15315315  661.66166166
  670.17017017  678.67867868  687.18718719  695.6956957   704.2042042
  712.71271271  721.22122122  729.72972973  738.23823824  746.74674675
  755.25525526  763.76376376  772.27227227  780.78078078  789.28928929
  797.7977978   806.30630631  814.81481481  823.32332332  831.83183183
  840.34034034  848.84884885  857.35735736  865.86586587  874.37437437
  882.88288288  891.39139139  899.8998999   908.40840841  916.91691692
  925.42542543  933.93393393  942.44244244  950.95095095  959.45945946
  967.96796797  976.47647648  984.98498498  993.49349349 1002.002002
 1010.51051051 1019.01901902 1027.52752753 1036.03603604 1044.54454454
 1053.05305305 1061.56156156 1070.07007007 1078.57857858 1087.08708709
 1095.595

In [36]:
plt.plot(Hs_abs, np.rad2deg(ang1))
plt.plot(Hs_abs, np.rad2deg(ang2))
plt.grid(True)
plt.xlabel("H Oe")
plt.ylabel("tet")

Text(0, 0.5, 'tet')

In [34]:
np.savetxt("angles1.csv", np.rad2deg(ang1))
np.savetxt("angles2.csv", np.rad2deg(ang2))
np.savetxt("Hs.csv", Hs_abs)

In [39]:
np.rad2deg(ang2)

array([178.02357781, 177.98893094, 177.95424719, 177.91953066,
       177.88477745, 177.84999011, 177.81516814, 177.78030917,
       177.74541705, 177.71048865, 177.67552485, 177.64052822,
       177.60549376, 177.57042542, 177.5353197 , 177.50018091,
       177.46500571, 177.42979455, 177.39454813, 177.35926562,
       177.32394811, 177.28859301, 177.25320405, 177.21777942,
       177.18231659, 177.1468215 , 177.11128936, 177.07571816,
       177.04011431, 177.00447376, 176.96879436, 176.93308091,
       176.89733077, 176.86154484, 176.82572257, 176.78986384,
       176.75396926, 176.71803641, 176.68206816, 176.64606215,
       176.61001923, 176.57394307, 176.53782634, 176.5016745 ,
       176.46548551, 176.42925899, 176.39299717, 176.35669576,
       176.32036101, 176.28398521, 176.24757523, 176.21112517,
       176.17463912, 176.13811626, 176.10155506, 176.06495592,
       176.02832164, 175.99164875, 175.95493733, 175.91818963,
       175.88140322, 175.84457904, 175.80771665, 175.77